In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [2]:
NOR_PATH = '../Machine_Learning/Data/202212/누수센서1_NOR/'
LEAK_PATH = '../Machine_Learning/Data/202212/누수센서1_누수1/'
LEAK_PATH2 = '../Machine_Learning/Data/202212/누수센서1_누수1_3/'

file_list_nor = os.listdir(NOR_PATH)
file_list_nor_csv = [file for file in file_list_nor if file.endswith('.csv')]
nor_length = len(file_list_nor_csv)
#print(len(file_list_nor_csv))

file_list_leak = os.listdir(LEAK_PATH)
file_list_leak_csv = [file for file in file_list_leak if file.endswith('.csv')]
leak_length = len(file_list_leak_csv)
#print(len(file_list_leak_csv))     

file_list_leak2 = os.listdir(LEAK_PATH)
file_list_leak_csv2 = [file for file in file_list_leak2 if file.endswith('.csv')]
leak_length2 = len(file_list_leak_csv2)

In [3]:
leak_length2

5977

In [4]:
a = pd.read_csv(NOR_PATH + file_list_nor_csv[0])
val = a['0.000000'].values
a

,0.000000,Unnamed: 1
0,12.292628,NaN
1,27.311138,NaN
2,44.465237,NaN
3,23.664658,NaN
4,25.087404,NaN
...,...,...
494,5.156961,NaN
495,24.918333,NaN
496,19.549744,NaN
497,10.368819,NaN


In [5]:
csv_list = []
for i in range(nor_length):
    #print(file_list_nor_csv[i])
    a = pd.read_csv(NOR_PATH + file_list_nor_csv[i])
    val = a['0.000000'].values
    csv_list.append(val)

df_nor = pd.DataFrame(csv_list)
nor_list = [0 for i in range(nor_length)]
#print(len(nor_list))
df_nor['leaktype'] = nor_list
#df_nor

csv_list = []
for i in range(leak_length2):
    a = pd.read_csv(LEAK_PATH + file_list_leak_csv2[i])
    val = a['0.000000'].values
    csv_list.append(val)

df_laek = pd.DataFrame(csv_list)
leak_list = [1 for i in range(leak_length)]
df_laek['leaktype'] = leak_list

df_nor.to_csv('../noleak.csv', index=False)
df = pd.concat([df_nor, df_laek])
df

df.to_csv('../airpoint202212_3nor.csv', index=False)

y = df['leaktype']
X = df.drop(['leaktype'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=11)

print(X_test)

#df_laek

            0          1          2          3          4          5    \
1083  21.758139  24.957499  27.138397  18.249784  16.271027   2.714837   
4658  13.699166  13.971316  24.960541  20.264109  17.299459  15.435134   
51    11.408278  19.604727  26.526215  28.284809  11.048405  20.700230   
5775   4.379516  12.449743  26.145582  19.182220  34.025192  32.272831   
5151  21.710304   9.181003   9.388799  26.705112  32.385906  14.112518   
...         ...        ...        ...        ...        ...        ...   
4913  14.409272  21.275751  11.515083   4.930369  21.421215  36.076534   
4255   8.442801  17.595530  26.086081  27.663517   9.183738   2.891371   
749   23.517317  17.617310  24.681503   9.980052  38.489250  13.039341   
4187   8.048556   6.362969  11.165330  15.271453  13.947737  34.500008   
4180   7.689169  33.416748  25.237373  12.737471  32.589157   4.359423   

            6          7          8          9    ...        489        490  \
1083  25.732063  31.953655   8.5

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time
import joblib

def models_predict(models):
    for model in models:
        start_time = time.time()
        model.fit(X_train, y_train)
        model_pred = model.predict(X_test)

        print('{0} 수행시간: {1:.3f}초'.format(model.__class__.__name__, time.time() - start_time))
        print('{0} 정확도:{1:.4f}'.format(model.__class__.__name__, accuracy_score(y_test, model_pred)))
        print('Normal', 'Leak')
        print(confusion_matrix(y_test, model_pred), '\n')
        print(classification_report(y_test, model_pred, target_names=['Normal', 'Leak']), '\n')
        
        joblib.dump(model, '../20221229.pkl')

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings('ignore')

dt_clf = DecisionTreeClassifier(random_state=11)

knn_clf = KNeighborsClassifier(n_neighbors=8)

rf_clf = RandomForestClassifier(random_state=11)

etc = ExtraTreesClassifier(random_state=11)

lgbm_wrapper = LGBMClassifier(random_state=11)

vo_clf = VotingClassifier(estimators=[('DT', dt_clf), ('RF',rf_clf), ('LGBM', lgbm_wrapper)], voting='soft')

#classifiers1 = [dt_clf, knn_clf, rf_clf, etc, lgbm_wrapper, vo_clf]
classifiers1 = [lgbm_wrapper]
models_predict(classifiers1)

LGBMClassifier 수행시간: 3.053초
LGBMClassifier 정확도:0.9850
Normal Leak
[[1190   18]
 [  18 1178]] 

              precision    recall  f1-score   support

      Normal       0.99      0.99      0.99      1208
        Leak       0.98      0.98      0.98      1196

    accuracy                           0.99      2404
   macro avg       0.99      0.99      0.99      2404
weighted avg       0.99      0.99      0.99      2404
 



In [8]:
import pandas as pd

a = pd.read_csv('airpoint.csv')
a = a.drop(['leaktype'], axis=1)
a

,0,1,2,3,4,5,6,7,8,9,...,489,490,491,492,493,494,495,496,497,498
0,36.345695,16.245274,11.260436,27.760191,19.124212,34.088528,10.397052,42.628830,43.828949,11.439801,...,32.818638,27.070524,2.457168,8.473702,15.272036,22.084114,9.744306,11.658361,4.613668,17.315752
1,15.790095,15.192574,9.966244,14.474835,27.273104,24.431196,3.715954,31.655800,4.973100,32.659229,...,13.951179,11.278485,14.026999,17.802212,26.923010,6.721580,38.562885,27.608755,8.473424,10.343286
2,15.601612,18.304953,23.552498,29.275059,33.345963,13.547577,29.820753,13.137032,2.410517,25.969662,...,15.573009,14.549078,8.010542,30.619581,12.382071,28.522665,20.634130,4.595755,28.722149,6.387628
3,19.923815,18.792496,5.050715,30.484159,30.640898,13.361160,35.136692,24.678526,12.141919,7.174418,...,23.630575,15.687149,46.585407,16.632948,18.753197,29.793543,24.416201,26.044868,11.022353,19.460581
4,95.226234,103.022736,131.581314,140.113739,31.098606,31.920677,61.406590,75.826294,23.719599,30.783504,...,19.232660,27.996403,47.085682,31.500818,24.790918,33.282871,43.120136,35.268185,27.895725,21.769531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,63.808708,23.182257,11.255012,27.954220,26.464764,25.999310,22.239096,8.706513,24.806736,4.665488,...,18.556831,8.205945,15.502442,43.804150,19.267557,13.068094,26.871317,29.542437,22.982901,23.756390
360,26.051237,14.467891,32.431458,13.172321,41.096455,16.578520,30.464096,12.137078,24.726488,26.119318,...,4.730232,18.450878,5.214909,15.084227,19.843420,41.986446,18.049631,5.960722,12.175860,17.658884
361,11.409794,8.416214,20.083754,11.157315,23.342405,22.834333,19.816307,8.349866,14.759519,13.452602,...,22.691408,23.806337,19.532522,15.905867,30.158314,17.610329,14.108180,6.196572,14.924394,4.912528
362,30.658171,17.657734,23.774385,20.056135,2.785266,33.484711,16.809614,16.537052,12.356853,30.943459,...,15.436600,31.898556,17.140942,24.722849,22.879538,21.518845,18.465725,13.265952,9.802721,10.971518
